# Deployment

## Concepts

`Langgraph`
- Python and JS library -> Creates Agentic Workflows

`Langgraph API`
- Bundles the code
- Provides a task queue for managing asynchronous operations
- Offers persistence for maintaining state across interactions

`Langgraph Cloud`
- Hosts the API
- Allowd graph deployment from Github repos
- Provides monitoring and tracing for deployed graphs
- Accessible via a unique url for each deployment

`Langgraph Studio`
- Integrated Development Evnironment (IDE) for Langgraph applications
- Uses API as backend, allows realtime testing and exploration of graphs
- Can be run locally, or with cloud-deployment

`Langgraph SDK`
- Python library for programmatically interacting with LangGraph graphs
- Provides a consistent interface for working with graphs, whether served locally or in the cloud
- Allows creation of clients, access to assistants, thread management, and execution of runs